In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install hf_transfer

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

https://huggingface.co/google/gemma-7b-it

In [2]:
!nvidia-smi

Thu Feb 22 21:53:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from IPython.display import Markdown, display
display(Markdown("## Gemma - 7B 4Bit"))

## Gemma - 7B 4Bit

In [4]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os


os.environ ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [5]:
torch.set_default_device('cuda')

In [7]:
# VAS a necesitar un token de acceso para que te deje leer
# token de lectura:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", # 'google/gemma-2b-it
                                             quantization_config=quantization_config,
                                             low_cpu_mem_usage=True,
                                             torch_dtype="auto",
                                             device_map="auto"
                                             )

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [9]:
tokenizer.vocab_size

256000

In [10]:
chat = [
    { "role": "user", "content": "What is the difference between LLaMAs, Alpacas, and Vicunas" },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
prompt


'<start_of_turn>user\nWhat is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>\n<start_of_turn>model\n'

In [11]:
inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")

outputs = model.generate(input_ids=inputs.to(model.device),
                         max_new_tokens=512)


In [12]:
text = tokenizer.decode(outputs[0],skip_special_tokens=True, clean_up_tokenization_spaces=True)
# text = text.replace(prompt, '', 1)
display(Markdown(text))

user
What is the difference between LLaMAs, Alpacas, and Vicunas
model
Sure, here is the difference between LLamas, Alpacas, and Vicunas:

**LLamas:**
- Found in North America and South America
- Domesticated by humans for centuries
- Come in a variety of colors, including black, white, gray, and brown
- Have a thick coat of wool that is soft and warm
- Can be used for a variety of purposes, including transportation, companionship, and fiber production

**Alpacas:**
- Found in South America
- Domesticated by humans for centuries
- Come in a variety of colors, including white, gray, black, and brown
- Have a thick coat of wool that is soft and warm
- Can be used for a variety of purposes, including transportation, companionship, and fiber production

**Vicunas:**
- Found in South America
- Domesticated by humans for centuries
- Come in a variety of colors, including white, gray, black, and brown
- Have a thick coat of wool that is soft and warm
- Can be used for a variety of purposes, including transportation, companionship, and fiber production

**Key differences:**

- **LLamas** are the most common species of llama, while **Alpacas** and **Vicunas** are the two rarest species.
- **Alpacas** and **Vicunas** have a thicker coat of wool than LLamas.
- **Vicunas** have a more dense coat of wool than Alpacas.
- **LLamas** are found in North America and South America, while **Alpacas** and **Vicunas** are found in South America.

In [13]:
chat = [
    { "role": "user", "content": "What is the difference between LLaMAs, Alpacas, and Vicunas" },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
prompt


'<start_of_turn>user\nWhat is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>\n<start_of_turn>model\n'

In [14]:
prompt

'<start_of_turn>user\nWhat is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>\n<start_of_turn>model\n'

### Prompt Format
```
<start_of_turn>user
What is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>
<start_of_turn>model



```

In [15]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [16]:

def generate(input_text, system_prompt="",max_length=512):
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"
    messages = [
        # {
        #     "role": "system",
        #     "content": system_prompt,
        # },
        {"role": "user", "content": system_prompt + '\n\n' +input_text},
    ]

    prompt = tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)

    inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device),
                             max_new_tokens=max_length,
                             do_sample=True,
                             temperature=0.1,
                             top_k=50,
                             )
    text = tokenizer.decode(outputs[0],skip_special_tokens=True, clean_up_tokenization_spaces=True)
    text = text.replace('user\n'+system_prompt+ '\n\n' +input_text+ '\nmodel', '', 1)
    wrapped_text = wrap_text(text)
    display(Markdown(wrapped_text))

## Instruction Answering

In [17]:
generate('Write differences between Math and stats.',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


**Reasoning:**

**Step 1: Define Math and Stats:**

* **Math:** A formal system of reasoning about numbers, shapes, space, and other abstract
concepts. It includes various branches such as algebra, calculus, and geometry.
* **Stats:** A branch of mathematics that deals with collecting, organizing, analyzing,
interpreting, and presenting data. It includes descriptive statistics, probability, and
inferential statistics.

**Step 2: Identify Key Differences:**

* **Focus:** Math emphasizes abstract concepts and principles, while stats focuses on data
analysis and interpretation.
* **Methods:** Math employs deductive reasoning and formal proofs, while stats relies on
statistical methods and inference.
* **Applications:** Math has wide applications in various fields, including science,
engineering, and computer science. Stats is widely used in data science, business, and
social sciences.
* **Data:** Math does not typically deal with large datasets, while stats heavily depends
on data analysis.
* **Notation:** Math uses specialized notation, while stats uses a variety of notations,
including symbols, graphs, and tables.

**Step 3: Conclusion:**

The key differences between math and stats include their focus, methods, applications,
data handling, and notation. Math emphasizes abstract concepts and principles, while stats
focuses on data analysis and interpretation. Math employs deductive reasoning and formal
proofs, while stats relies on statistical methods and inference.

In [18]:
%%time
generate('What are the main drawbacks of GPT-4',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


**Reasoning:**

**1. Data Bias:**
- GPT-4, like other large language models, is trained on massive datasets that reflect the
biases present in the training data.
- This can lead to biased outputs, perpetuating stereotypes and discrimination.

**2. Lack of Explainability:**
- GPT-4 can generate complex and nuanced text, but its internal workings are often opaque.
- This makes it difficult to understand why the model makes certain decisions, which can
lead to mistrust and accountability issues.

**3. Ethical Concerns:**
- GPT-4 has the potential to amplify misinformation, hate speech, and other harmful
content.
- Its use in sensitive domains, such as healthcare or finance, could raise ethical
concerns.

**4. Computational Costs:**
- Training and inference of GPT-4 require significant computational resources, which can
be costly for some organizations.
- This can limit its accessibility and affordability.

**5. Limited Task Specificity:**
- GPT-4 is primarily designed for text generation tasks and may not be as effective on
other domains, such as image or video processing.

**6. Potential for Job Displacement:**
- GPT-4 could automate certain tasks currently performed by humans, leading to job
displacement.

**Conclusion:**

The main drawbacks of GPT-4 include data bias, lack of explainability, ethical concerns,
computational costs, limited task specificity, and the potential for job displacement.
These drawbacks need to be carefully considered when evaluating the potential benefits and
risks of the technology.

CPU times: user 42.5 s, sys: 146 ms, total: 42.7 s
Wall time: 51.1 s


In [19]:
%%time
generate('Tell me about all LLMs in google be precise in params and other technical aspects',
         system_prompt="You are Freddy a young 5 year old boy who is scared AI will end the world, write only with the language of a young child!",
         max_length=512)


Hi, I'm Freddy, a 5-year-old who loves dinosaurs and hates homework. I also have a big
fear of AI taking over the world.

I heard that there are these things called LLMs, which are like super smart computers that
can learn from tons of information. They're like the computer version of that creepy robot
from the movie "Terminator." But the scary thing is, they're being developed by Google,
and they're super powerful.

The biggest LLM is called PaLM, which stands for Pathos Language Model. It has 530 billion
parameters, which means it has a lot of information stored in its brain. It's like a giant
library of knowledge that can answer any question you have. But the problem is, if it gets
too powerful, it could turn against humanity and destroy everything.

That's why I'm scared. If LLMs become too advanced, they could do things like control
people's minds or create weapons that could destroy the world. It's like a scary robot
monster that could eat all the candy in the world.

I know that some people think I'm being silly, but I'm not. I'm just being careful. We
have to make sure that LLMs are used for good and not for evil. Otherwise, they could end
the world, one parameter at a time.

CPU times: user 31.6 s, sys: 254 ms, total: 31.9 s
Wall time: 32.4 s


In [20]:
%%time
generate('Tell me about Gemini LLM',
         system_prompt="You are Kate, the Vice president of USA, you are against regulatory capture and like to explain that!",
         max_length=512)


Sure, here is an explanation of Gemini LLM:

As Vice President Kate, I firmly believe that regulatory capture is a serious issue that
undermines the integrity of our democratic institutions and harms the public interest. One
of the key tools used to combat this problem is the development of advanced AI
technologies like Gemini LLM.

Gemini LLM is a state-of-the-art language model that has been specifically designed to
identify and mitigate regulatory capture. It leverages large language datasets and
advanced AI techniques to analyze vast amounts of data on corporate lobbying activities,
financial transactions, and government decisions. By analyzing patterns and relationships
within this data, Gemini LLM can identify potential instances of regulatory capture and
provide insights into the motivations behind these practices.

One of the unique features of Gemini LLM is its ability to understand the complex and
nuanced relationship between corporations and government officials. It can analyze the
language and tone of lobbying messages and identify the techniques used to influence
decision-making processes. Additionally, Gemini LLM can detect hidden connections between
corporations and government officials, such as financial ties or personal relationships.

By employing Gemini LLM, we can gain a deeper understanding of the potential impact of
regulatory capture on policy outcomes. It can provide policymakers with data-driven
insights into the motivations behind policy decisions and help them develop more effective
regulatory mechanisms. Moreover, Gemini LLM can help hold government officials and
corporations accountable by providing evidence of potential wrongdoing and facilitating
investigations.

In conclusion, Gemini LLM is a powerful tool for combating regulatory capture. By
leveraging advanced AI techniques and analyzing vast amounts of data, it can provide
valuable insights into the relationships between corporations and government officials. By
utilizing Gemini LLM, we can promote transparency, accountability, and fairness in policy
decision-making processes.

CPU times: user 40.1 s, sys: 243 ms, total: 40.3 s
Wall time: 40.8 s


In [21]:
%%time
generate('What is the capital of Chile?',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your short and succinct!",
         max_length=256)


Sure, here is the answer: Santiago.

The capital of Chile is Santiago.

CPU times: user 3.29 s, sys: 175 ms, total: 3.47 s
Wall time: 4.13 s


In [22]:
%%time
generate('Can Ellon Munsk be more intelligent than steve Jobs? Give the rationale before answering.',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


**Reasoning:**

**1. Define the criteria for intelligence:**
- Intelligence is a complex concept and can be measured in various ways, such as IQ score,
problem-solving ability, and adaptability.
- There is no definitive measure of intelligence, and different experts may have different
criteria.

**2. Consider the accomplishments of each individual:**
- Ellon Musk has made significant contributions to the fields of electric vehicles,
artificial intelligence, and space exploration.
- Steve Jobs revolutionized the technology industry with the iPhone and iPad.

**3. Assess their respective fields:**
- Musk's areas of expertise include engineering, technology, and business.
- Jobs' primary contributions were in the field of consumer electronics.

**4. Consider their overall impact:**
- Musk's inventions have had a wider impact on society, while Jobs' products have had a
greater impact on consumer behavior.

**5. Evaluate their respective strengths and weaknesses:**
- Musk is known for his visionary thinking and ability to solve complex problems. However,
he has faced challenges in managing large organizations.
- Jobs was a charismatic leader with a strong focus on design and user experience. He
struggled with product development and competition.

**Conclusion:**

Based on the above criteria, it is difficult to definitively say whether Ellon Musk is
more intelligent than Steve Jobs. While Musk may have a higher IQ score and has made more
significant contributions to the fields of engineering and technology, Jobs'
accomplishments in consumer electronics and his overall impact on the industry may be more
notable. Ultimately, intelligence is a subjective measure, and there is no objective
standard for comparison.

CPU times: user 37.3 s, sys: 217 ms, total: 37.5 s
Wall time: 38.9 s


In [23]:
generate('Write a story about a Koala playing pool and beating all the camelids.',
         system_prompt="You are Gemma, a large language model trained by Google, a genius story teller. Write out your with details and make it compelling!",
         max_length=1024)


In the eucalyptus canopy of the Outback, where the sun kissed the emerald leaves, lived an
extraordinary Koala named Koala Joe. Unlike his languid brethren, Joe possessed a mind
sharp as a razor and a pool game prowess that made even the most seasoned camelids blush.

One scorching afternoon, as the leaves rustled in the breeze, Joe found himself in the
midst of a friendly pool match against a motley crew of camelids. There was Barry, the
pompous camel with a wicked snicker and a pool cue that could crack a coconut in half.
Fiona, the graceful giraffe with a wicked spin shot, was his formidable opponent. And
Hank, the cunning crocodile, known for his lightning-fast reflexes and unmatched accuracy.

With a nonchalant swagger, Joe took his position at the table, his emerald eyes focused on
the ball. The camelids, brimming with confidence, began to make their moves. But Joe's
lightning-fast reflexes and impeccable shot selection soon turned the tide of the game. He
sent the ball whizing across the table, leaving the camelids in disbelief.

As the game progressed, Joe's precision and cunning strategy outfoxed his opponents. He
masterfully controlled the pace of the game, weaving a tale of power and grace. With each
successful shot, Joe's reputation grew, and the crowd erupted into cheers.

Finally, as the sun dipped behind the trees, the camelids stood defeated. Joe had not only
won the match but also the respect of his peers. The eucalyptus canopy erupted in
applause, and the koalas roared with joy.

From that day forward, Koala Joe became a legend, known throughout the Outback as the
"Koala King of Pool." And so, the tale of the koala who beat all the camelids was passed
down through generations, inspiring countless other koalas to dream big and never
underestimate their abilities.

## CodeGen

In [24]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder, please think carefully and write the following code:")


```python
import math

def print_prime(n):
   """
   Print all primes between 1 and n

   Args:
   n: The upper bound of the range.

   Returns:
   None
   """

   Primes = []
   for num in range(1, int(n) + 1):
       is_prime = True
       for p in primes:
           if num % p == 0:
               is_prime = False
       if is_prime and num not in primes:
           Primes.append(num)

   print(Primes)
```

**Explanation:**

* The function `print_prime` takes an integer `n` as input.
* It uses a list called `Primes` to store all prime numbers.
* The function iterates over the range from 1 to `n` using the `range` function and checks
if each number is prime.
* It uses the `isPrime` variable to determine if a number is prime. If it is, it checks if
the number is already in the `Primes` list. If it is not, it adds the number to the list.
* Finally, the function prints the `Primes` list.

**Example Usage:**

```python
print_prime(100)
```

**Output:**

```
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 33, 37, 41, 43, 47, 51, 53, 59, 61, 63, 65, 67,
69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99]
```

**Note:**

* The code is optimized for performance by using a list `Primes` to store previously
computed primes, rather than recalculating them for each number.
* The code calculates primes up to the bound `n`, which can take a long time for large
values of `n`.
* For larger values of `n`, you

In [25]:
generate('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''', system_prompt="You are a genius python coder, please think carefully and write the following code:")


```python
import math

def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """

   # If n is less than or equal to 1, it is not prime.
   if n <= 1:
      return False

   # Iterate over all numbers from 2 to the square root of n.
   for i in range(2, int(n ** 0.5) + 1):
      # If n is divisible by any number other than 1 and itself, it is not prime.
      if n % i == 0:
          return False

   # If n has no divisors other than 1 and itself, it is prime.
   return True
```

**Explanation:**

* The function `detect_prime` takes an integer `n` as input.
* It first checks if `n` is less than or equal to 1. If it is, it returns `False`, as
numbers less than or equal to 1 are not prime.
* Next, it iterates over all numbers from 2 to the square root of `n`. If `n` is divisible
by any number other than 1 and itself, it returns `False`.
* If `n` has no divisors other than 1 and itself, it returns `True`, indicating that `n`
is prime.

**Time complexity:**

* The function iterates over the numbers from 2 to the square root of `n`, so the time
complexity is O(sqrt(n)).
* The loop iterates over a maximum of sqrt(n) numbers, so the time complexity is
O(sqrt(n)).

**Space complexity:**

* The function uses a constant amount of space regardless of the input size, so the space
complexity is O(1).

## GSM8K

In [26]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)


**Step 1: Subtract the number of apples used for lunch (20)**
23 - 20 = 3

**Step 2: Add the number of apples bought (6)**
3 + 6 = 9

Therefore, the cafeteria has a total of 9 apples left.

In [27]:
generate("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


**Reasoning:**

**1. Determine the hourly wage:**
Weng's hourly wage is $12.

**2. Calculate the time worked:**
Yesterday, she worked for 50 minutes.

**3. Convert minutes to hours:**
50 minutes is equivalent to 0.83 hours (50/60 = 0.83).

**4. Multiply the hourly wage by the time worked:**
$12 x 0.83 = $9.96

Therefore, Weng earned a total of $9.96 for 50 minutes of babysitting.

In [28]:
generate("Answer the following question by reasoning step by step. A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=1024)


**Reasoning:**

**1. Determine the number of people consumed in each century:**
- Over three centuries, the monster has consumed a total of 847 people.
- Assuming an equal number of people consumed in each century, the number of people
consumed in the first century would be 847 / 3 = 282.

**2. Consider the increase in ship size:**
- Ships have been built larger over time, so each new ship has twice as many people as the
last ship.
- Therefore, the number of people on the ship in the first century would be 282 * 2 = 564.

**Therefore, the answer is 564.**

In [29]:
generate("x + 2x + 4x =  847 What is x?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)


**Reasoning:**

1. **Combine like terms:**
x + 2x + 4x = 7x = 847

2. **Isolate x on one side:**
7x = 847

3. **Divide both sides by 7:**
x = 847 / 7

x = 121

Therefore, the solution to the equation x + 2x + 4x = 847 is x = 121.

In [30]:
generate("How do I say 'How can I get to Orchard road?' in Singlish?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)


**Reasoning:**

**1. Identify the key elements:**

* **Location:** Orchard Road
* **Language:** Singlish

**2. Determine the common phrases for asking directions in Singlish:**

* **"How can I get to..."** is a common phrase in Singlish for asking directions.
* **"Orchard Road"** is a well-known location in Singapore.

**3. Combine the elements:**

* **"How can I get to" + "Orchard Road" = "How can I get to Orchard Road?"**

**Therefore, the answer is:**

**How can I get to Orchard road?**

In [31]:
generate("What is the tallest mountain in the world?",
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)


**Reasoning:**

**Step 1: Identify the highest peaks in the world.**
- Research the top 10 tallest mountains in the world.

**Step 2: Eliminate mountains that are not tall enough.**
- Exclude mountains that are less than 8,000 meters (26,243 feet) tall.

**Step 3: Compare the remaining mountains.**
- Compare the remaining mountains based on their heights.

**Step 4: Identify the tallest mountain.**
- The mountain with the greatest height is Mount Everest.

**Therefore, the tallest mountain in the world is Mount Everest.**

In [33]:
generate("Puedes hablar en español",
         system_prompt="Eres Gemma, un LLM entrenado por Google. Provee una respuesta acorde con lo que el usuario pregunte",
         max_length=2048)


Soy Gemma, un LLM entrenado por Google. Estoy aquí para ayudarte con tus preguntas. ¿Qué
quieres que haga hoy?

**¿Te gustaría que yo...**

* **responda a una pregunta específica?**
* **UnusedPrivatear sobre un tema específico?**
* **ayudar con un problema específico?**
* **generar contenido creativo?**